In [93]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [94]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv') #讀資料夾底下的csv
    symbol=etf_csv['Symbol']#all etf
    inception=etf_csv['Inception'] # all etf inception time
    count=0
    real_symbol=[] #etf
    in_s=[] #time
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

In [95]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+ETF+"?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk' or freq == '1mo'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [96]:
etfs=get_etf()
etfs.remove('ZJPN')
etfs.remove('PAF')# 移除不合規定的etf
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target,etfs):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    for Symbol in etfs:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data

In [97]:
all_data_week = get_Date('1wk','adjclose',etfs)
all_data_month = get_Date('1mo','adjclose',etfs)

----------1wk----------
start crawling : EWJ
start crawling : EWY
start crawling : FNDF
start crawling : VPL
start crawling : DXJ
start crawling : EWT
start crawling : EWH
start crawling : EPP
start crawling : FNDC
start crawling : DLS
start crawling : JPIN
start crawling : PXF
start crawling : GSIE
start crawling : EWA
start crawling : FDT
start crawling : IPAC
start crawling : GWX
start crawling : DWM
start crawling : DFJ
start crawling : HEWJ
start crawling : DBJP
start crawling : EWS
start crawling : DOL
start crawling : HFXI
start crawling : PDN
start crawling : IVLU
start crawling : IDOG
start crawling : DTH
start crawling : DIM
start crawling : DOO
start crawling : SCJ
start crawling : ENZL
start crawling : IDLB
start crawling : FJP
start crawling : JPXN
start crawling : IVAL
start crawling : DXJS
start crawling : ISCF
start crawling : DEEF
start crawling : IMOM
start crawling : JPN
start crawling : IDHQ
start crawling : FPA
start crawling : DVYA
start crawling : JPMV
start craw

In [113]:
all_data_week=all_data_week.dropna()
#all_data_week

In [114]:
#all_data_month

In [115]:
all_data_month=all_data_month.dropna()
#all_data_month

In [116]:
def  calculate_ra_1(etf): # calculate one ra
    today_etf=etf[-1]
    return [(today_etf-etf[i])/etf[i] for i in range(0,len(etf)-1)]

In [117]:
def calculate_ra_all(date,etfs,data):
    dict={}
    for etf in etfs:
        dict[etf]=calculate_ra_1(data[etf])
    
    df = pd.DataFrame.from_dict(dict)
    return df

In [118]:
returns=calculate_ra_all(all_data_month.index,etfs,all_data_month)
returns

,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,DLS,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
0,0.240612,0.348672,0.323711,0.342096,0.144242,0.523363,0.616359,0.475556,0.323501,0.318936,...,0.263717,0.339626,0.293627,0.234725,0.515152,0.295901,0.439127,0.091797,0.347951,0.392645
1,0.310105,0.381962,0.360447,0.390904,0.307549,0.462855,0.605637,0.484789,0.343888,0.333523,...,0.303141,0.460890,0.293627,0.269124,0.487276,0.336656,0.395401,0.070148,0.419313,0.420284
2,0.250398,0.218963,0.263847,0.294016,0.245028,0.355200,0.469194,0.331086,0.237540,0.228245,...,0.250302,0.341204,0.224123,0.214279,0.394172,0.268436,0.284856,-0.026780,0.249740,0.370111
3,0.249303,0.232261,0.220017,0.285226,0.324092,0.438213,0.468451,0.306763,0.212645,0.214684,...,0.208859,0.360614,0.190641,0.204549,0.349955,0.312136,0.264252,-0.010869,0.250043,0.303601
4,0.210097,0.293923,0.238204,0.271714,0.239922,0.408026,0.470682,0.335854,0.196885,0.197177,...,0.278408,0.273852,0.190641,0.172146,0.352975,0.266792,0.291286,0.032422,0.266905,0.275300
5,0.240612,0.236520,0.259588,0.280614,0.403305,0.336873,0.483451,0.331765,0.228370,0.287856,...,0.324767,0.334567,0.190641,0.177610,0.324620,0.386452,0.254196,0.020244,0.210608,0.311849
6,0.180693,0.156766,0.217038,0.200844,0.321491,0.249705,0.362172,0.230051,0.163696,0.189694,...,0.266297,0.211078,0.188187,0.155923,0.238973,0.345240,0.169928,-0.008770,0.117978,0.253641
7,0.159541,0.142602,0.205265,0.198607,0.245933,0.249705,0.353752,0.260251,0.167384,0.187306,...,0.234654,0.173496,0.157034,0.141008,0.214109,0.373801,0.182898,-0.029369,0.127521,0.240864
8,0.135499,0.107614,0.186141,0.168335,0.254354,0.196374,0.293476,0.214050,0.131912,0.156545,...,0.206341,0.137671,0.153396,0.090517,0.175031,0.298945,0.184577,-0.048354,0.150065,0.258123
9,0.127408,0.163245,0.191815,0.175024,0.190736,0.191819,0.319861,0.251036,0.147584,0.173597,...,0.188978,0.135151,0.182714,0.094128,0.215917,0.237591,0.227615,0.023822,0.184101,0.245148


In [119]:
week_returns=calculate_ra_all(all_data_week.index,etfs,all_data_week)
week_returns

,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,DLS,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
0,0.168329,0.318812,0.265827,0.258216,0.084517,0.442801,0.444800,0.350383,0.256738,0.258402,...,0.150914,0.147212,0.295951,0.188428,0.177459,0.111498,0.354271,0.070157,0.306925,0.364219
1,0.237775,0.401784,0.358198,0.345849,0.181737,0.572062,0.553768,0.489262,0.339280,0.336132,...,0.228299,0.282469,0.295951,0.235163,0.296207,0.186745,0.407279,0.100678,0.465340,0.295436
2,0.266560,0.439995,0.399814,0.386411,0.218592,0.624741,0.613292,0.547038,0.388780,0.374824,...,0.266161,0.375406,0.295951,0.235163,0.349221,0.256884,0.363989,0.122529,0.492718,0.390427
3,0.254220,0.384306,0.365580,0.362306,0.177893,0.560082,0.628893,0.500904,0.371691,0.364051,...,0.253089,0.343084,0.214421,0.267695,0.375474,0.287432,0.432538,0.154510,0.490297,0.390427
4,0.231317,0.372127,0.334750,0.338020,0.141981,0.494288,0.593541,0.465269,0.335003,0.332965,...,0.259554,0.322368,0.295951,0.225525,0.374042,0.287432,0.416413,0.117884,0.368742,0.395440
5,0.293164,0.377900,0.368058,0.388300,0.243706,0.487051,0.656214,0.502209,0.372253,0.361257,...,0.288720,0.431863,0.295951,0.241897,0.418350,0.186262,0.387016,0.065001,0.518625,0.405645
6,0.360244,0.409627,0.408972,0.436694,0.364296,0.517674,0.651438,0.514497,0.412835,0.401977,...,0.415401,0.586343,0.295951,0.334595,0.416827,0.408829,0.439923,0.115320,0.463589,0.455614
7,0.307493,0.404790,0.365580,0.392911,0.315394,0.455337,0.607857,0.465269,0.369444,0.359738,...,0.340674,0.468871,0.295951,0.270394,0.357550,0.304144,0.390883,0.101675,0.497584,0.395164
8,0.280302,0.414803,0.361879,0.378370,0.268690,0.440545,0.580350,0.474857,0.353372,0.347956,...,0.298849,0.400929,0.295951,0.238238,0.351063,0.327921,0.374391,0.094731,0.438948,0.423133
9,0.229179,0.316955,0.289649,0.302816,0.213419,0.358744,0.497695,0.353203,0.293687,0.285619,...,0.298849,0.302280,0.295951,0.213071,0.248381,0.247734,0.305350,0.041284,0.361138,0.352876


In [120]:
# returns=calculate_ra_all(all_data_month.index,etfs)
# returns

In [122]:
def OmegaSharpeRatio(R, L):
    L=L/100
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [127]:
def ranking(etfs,returns,date):
    arr=[]
    symbols=[]
    if date=='week':
        rf=2.24/4
    for etf in etfs:
        if OmegaSharpeRatio(returns[etf],rf)!="undefined":
            arr.append(OmegaSharpeRatio(returns[etf],rf))
            symbols.append(etf)
    dict_omega={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_omega)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    return df

In [128]:
data=ranking(etfs,week_returns,'week')
data

,ETF_ Symbol,Value
1,HAUD,2883.096131
2,EWH,817.184182
3,EPP,74.793238
4,SCIX,29.360763
5,EWA,25.142955
6,DOO,21.003610
7,FAUS,16.532639
8,IDHQ,13.957150
9,HFXI,13.154458
10,IDOG,11.149679


In [129]:
data=ranking(etfs,returns,'month')
data

TypeError: ranking() missing 1 required positional argument: 'date'